In [2]:
import numpy as np
import os


In [5]:
import numpy as np

feature = np.random.rand(5, 1250, 5000)
universe = np.random.rand(1250, 5000)
universe = np.where(universe > 0.8, 0, 1)

In [17]:
import numpy as np

feature = np.random.rand(5, 1250, 5000)
universe = np.random.rand(1250, 5000)
universe = np.where(universe > 0.8, 0, 1)

timestep = 10
res = np.full(shape=(universe.shape[0], universe.shape[1], timestep), fill_value=-1, dtype=int)

def main():
    for di in range(timestep-1, universe.shape[0]):
        ix = universe[di] == 1
        for ii in range(universe.shape[1]):
            if not ix[ii]:
                res[di, ii] = res[di - 1, ii]
            # 在前面universe不为0的日子中搜索该股票，找不到就保持-1
            # 使用np.where查找universe中的非零值
            idxs = np.where(universe[:di+1, ii] == 1)[0]
            if len(idxs) > 0:
                # 获取最近的`timestep`个非零值的日期
                if len(idxs) >= timestep:
                    recent_idxs = idxs[-timestep:]
                else:
                    recent_idxs = idxs[-len(idxs):]
                res[di, ii, -len(recent_idxs):] = recent_idxs
                # 更新res数组，如果recent_idxs长度小于timestep，则剩余部分保持-1                   
    return res
res = main()

In [27]:
di = 1000

feature[:, di, :].shape

(5, 5000)

In [26]:
res[di, :, :].shape

(5000, 10)

In [21]:
universe[-20:, 2]

array([1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1])

In [32]:
def get_feature_slices_for_all_stocks(feature, res, di):
    # 获取di对应的所有股票的日期索引
    date_indices = res[di, :, :]
    
    # 初始化结果数组，形状为(feature_nums, ii, timestep)，默认值为0
    feature_slices = np.zeros((feature.shape[0], feature.shape[2], timestep))
    
    # 对于每个时间步，进行向量化操作
    for t in range(timestep):
        # 获取当前时间步的日期索引
        current_date_indices = date_indices[:, t]
        
        # 过滤掉值为-1的索引
        valid_indices = current_date_indices != -1
        
        # 获取有效的日期索引和股票索引
        valid_date_indices = current_date_indices[valid_indices]
        valid_ii_indices = np.arange(len(valid_indices))[valid_indices]
        
        # 使用有效的日期索引和股票索引从feature中提取切片
        feature_slices[:, valid_ii_indices, t] = feature[:, valid_date_indices, valid_ii_indices]
        
    return feature_slices

# 测试函数
di = 20  # 例子中的某个日期索引

for di in range(universe.shape[0]):
    feature_slices = get_feature_slices_for_all_stocks(feature, res, di)

In [31]:
feature_slices

array([[[8.34251540e-02, 1.16300614e-01, 4.89524770e-01, ...,
         8.49415457e-01, 3.39766813e-01, 1.45574938e-01],
        [4.97410976e-01, 3.94108145e-02, 2.27768511e-01, ...,
         7.23426137e-01, 8.26877033e-01, 2.71769693e-01],
        [2.93667247e-01, 5.89127745e-01, 2.07982266e-01, ...,
         4.36392809e-01, 2.06354086e-01, 9.92089700e-01],
        ...,
        [8.09199095e-01, 7.57303927e-01, 6.74149034e-01, ...,
         2.58585861e-01, 2.50219504e-01, 1.84966481e-01],
        [6.87832439e-01, 6.09680448e-02, 9.07762950e-01, ...,
         2.79824940e-01, 7.18229861e-01, 5.65848704e-01],
        [5.43452331e-02, 9.25778507e-01, 5.75648155e-01, ...,
         4.96651352e-01, 8.46122904e-01, 3.03510609e-01]],

       [[4.23735799e-01, 3.86030163e-01, 7.39162073e-01, ...,
         1.98505588e-01, 9.26125731e-01, 3.54386249e-01],
        [1.35330940e-04, 4.25253431e-01, 3.06364628e-01, ...,
         4.30008387e-01, 1.53800524e-01, 7.96258756e-01],
        [8.01950391e-01, 